## 准备数据

In [49]:
import numpy as np

def file_to_matrix(filename):
    """
    读取文件
    filename:文件名
    return:
        dataX--特征矩阵
        dataY--标签
    """
    # 读文件
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    #获取数据条数
    n = len(lines)
    dataX = np.zeros((n, 3))
    dataY = []
    
    for index, line in enumerate(lines):
        # 去空白
        line = line.strip()  
        list_of_line = line.split('\t')
        # 前三个值放入dataX
        dataX[index, :] = [float(i) for i in list_of_line[:-1]]
        if(list_of_line[-1]=="didntLike"):
            dataY.append(1)
        elif (list_of_line[-1]=="smallDoses"):
            dataY.append(2)
        else:
            dataY.append(3)
    
    return dataX,np.array(dataY)
            
            

In [50]:
dataX ,dataY =file_to_matrix("datingTestSet.txt")


In [57]:
# me
def read_file(file_name):
    dataX=[]
    dataY=[]
    with open(file_name,'r') as f:
        for line in f:
            # 去空白
            line = line.strip() 
            cnt = line.split('\t')
            dataX.append(cnt[:-1])
            dataY.append(cnt[-1])
    dataX = np.array(dataX)
    Y=[]
    # 转换标签
    for i in dataY:
        if i == "didntLike":
            Y.append(1)
        elif i=="smallDoses":
            Y.append(2)
        else:
            Y.append(3)
    return dataX,np.array(Y)
    
dataX ,dataY =read_file("datingTestSet.txt")


In [51]:
# 最大最小规范化

def auto_norm(dataX):
    """规范化处理
    参数 dataX矩阵
    
    标准化后的数据
    """
    
    max_vector = dataX.max(0)
    min_vector = dataX.min(0)
    n = len(dataX)
    max_mat = np.tile(max_vector,(n,1))
    min_mat = np.tile(min_vector,(n,1))
    
    normX = (dataX-min_mat)/(max_mat-min_mat)
    
    return normX

auto_norm(dataX)

array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629],
       ...,
       [0.29115949, 0.50910294, 0.51079493],
       [0.52711097, 0.43665451, 0.4290048 ],
       [0.47940793, 0.3768091 , 0.78571804]])

In [38]:
# me
def max_min_score(data_x):
    max = data_x.max(0)
    min = data_x.min(0)
    
    maxmin = (data_x-min)/(max-min)
    return maxmin

max_min_score(dataX)

array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629],
       ...,
       [0.29115949, 0.50910294, 0.51079493],
       [0.52711097, 0.43665451, 0.4290048 ],
       [0.47940793, 0.3768091 , 0.78571804]])

In [32]:
a= np.array([[1,2,3],[4,5,6]])
np.tile(a,(2,1))

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

## 找x的K近邻，并预测x的类别


In [55]:
import operator
def knn(x,train_x,train_y,k):
    """
    找x的k个近邻，并预测x的类型
    
    参数：
        x 要预测的样本
        train_x训练集的特征和矩阵
        train_y 训练集的标签向量
        k k个近邻
    返回值：预测的类别
    """
    n = len(train_x)
    #x_mat = np.tile(x,(n,3))
    # sum(0) 保留行    sum(0) 保留列
    d_vector =((x-train_x)**2).sum(1)**(0.5)
    # 对向量从小到大排序(根据坐标来排序)
    idxs = d_vector.argsort()
    # 统计k个样本的类别
    class_count = {}
    class_count[1]=0
    class_count[2]=0
    class_count[3]=0
    for i in range(k):
        i_class = train_y[idxs[i]] 
        class_count[i_class]+=1
    
    class_count=sorted(class_count,items(),
          key=operator.itemgetter(1),reverse=True)
    
    return class_count[0][0]


x = np.array([[0.44832535, 0.39805139, 0.56233353]])

k =1
knn(x,dataX,dataY,k)

NameError: name 'items' is not defined

In [58]:
# 书上的
def classify(intX,dataSet,labels,k):
    dataSetSize = dataSet.shape[0]
    diffMat = tile(intX,(dataSetSize,1))-dataSet
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis=1)
    sqDistances = sqDistances**(0.5)
    sortedDistIndicies = distances.argsort()
    classCount = []
    for i in range(k):
        voteiLabel = labels[sortedDistIndicies[i]]
        classCount[voteiLabel]=classCount.get(voteiLabel,0)+1
    sortedClassCount = sorted(classCount.iteritems(),
                             key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

x = np.array([[0.44832535, 0.39805139, 0.56233353],
       [0.15873259, 0.34195467, 0.98724416],
       [0.28542943, 0.06892523, 0.47449629]])
x = max_min_score(x)

knn(x,dataX,dataY,1)

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [ ]:
def test(norm_x,data_y,k):
    """测试并评价模型
    参数：
        normX 标准化后的特征矩阵
        dataY 标签
        k k近邻
    返回值：
        error_rate 错误率
    """
    
     n = len(normX)
    # 测试集样本的个数
    test_n = int(n*0.1)
    testX = normX[:test_n,:]
    testY = dataY[:test_n]
    trainX = normX[test_n,:]
    trainY = dataY[test_n:]
    error_num = 0
    
    for x,y in zip(testX,testY):
        y_hat = classify(x,trainX,trainY,k)
        if (y_hat!=y):
            error_num+=1
            
    error_rate = error_num/test_n
    return error_rate
            
    